# Smart Daily Scheduler Agent (Concierge Track)

The **Smart Daily Scheduler Agent** is a simple personal-workflow agent designed to help users manage their day efficiently. It takes user-defined tasks, events, classes, and deadlines along with constraints (like working hours or sleep hours) and generates a structured, readable daily schedule.

**Key Capabilities:**
- Accepts a list of tasks/events with optional preferred start times and durations.
- Enforces user-defined time constraints (e.g., working hours, sleep hours).
- Produces a structured schedule in a human-friendly text format.

## Motivation

Daily planning can be cumbersome and error-prone. Users often forget to account for:
- Overlapping tasks or events
- Daily time constraints (work, school, sleep)
- Proper sequencing of priorities

The Smart Daily Scheduler Agent aims to:
- Automate task scheduling
- Respect user-defined constraints
- Provide a clear, readable daily schedule

It demonstrates how AI can assist in routine personal productivity and scheduling tasks efficiently.

## Session & Pipeline Flow Diagram

*Mermaid Code:*
```
flowchart TD
    A[User Input: Free-text tasks/events] --> B[Task Intake Agent]
    B --> C[Session State: stores 'tasks']
    C --> D[Time Constraint Tool]
    D --> E[Session State: stores 'adjusted_tasks']
    E --> F[Schedule Generator Agent]
    F --> G[Session State: stores 'readable_schedule']
    G --> H[Output: Human-readable daily schedule]

    style A fill:#f9f,stroke:#333,stroke-width:2px,color:black
    style B fill:#fefefe,stroke:#333,stroke-width:2px,color:black
    style C fill:#fefefe,stroke:#333,stroke-width:1px,color:black
    style D fill:#bbf,stroke:#333,stroke-width:2px,color:black
    style E fill:#fefefe,stroke:#333,stroke-width:1px,color:black
    style F fill:#fb7,stroke:#333,stroke-width:2px,color:black
    style G fill:#fefefe,stroke:#333,stroke-width:1px,color:black
    style H fill:#ff9,stroke:#333,stroke-width:2px,color:black
```


[![](https://mermaid.ink/img/pako:eNqllM1u4jAUhV_Ful10ExhCIIAXI_HPLEazgNlMUlUG30AmiY1sZwpFvPsYk1TtAqmizsb2PefLsWX7BBvJESgkuXzZ7JgyZDWJBbFtGP3WqMgPsS8NJTOF2DB4MMQwnelv-A-F0U-k0fhORtHKzlmlYRmS4dZWnq6MkauPoyVqnUpBloYZpEQbqVCTR4d6rLRjp51Eq7RAMpZCG8VSYfNImVeSiZNMb-EY_1tqg_z5A3fqTLNoudkhL3MkcxSomPV8iDpzsvkttkLG2TrHZ11havzc-RbRr9K4jVqUBRONWk44S_MjqU3Wc3Vpc7TFIUnSPKcPySDx7HJlhvQhCIKq33hJudnR9v7gbWQuFV3nbJO9949qP16-uxDjTyL824hJhViv71vF9OsRZjVi3bsrwvzrERY1Ihl8OgJ4sFUpB2pUiR4UqAp2GcLpQo7B7LDAGKjtcqayGGJxtp49E3-kLGqbkuV2BzRhubajcs_twZ2kbKtY8TarUHBUY1kKA7TnOwbQExyAtvth0w96Hb8ftv3uoBt6cAQahM1W2O763U47bA3CTu_swav7aavZ73U9QJ7aq_Hz-nq4R-T8H-HTa5s?type=png)](https://mermaid.live/edit#pako:eNqllM1u4jAUhV_Ful10ExhCIIAXI_HPLEazgNlMUlUG30AmiY1sZwpFvPsYk1TtAqmizsb2PefLsWX7BBvJESgkuXzZ7JgyZDWJBbFtGP3WqMgPsS8NJTOF2DB4MMQwnelv-A-F0U-k0fhORtHKzlmlYRmS4dZWnq6MkauPoyVqnUpBloYZpEQbqVCTR4d6rLRjp51Eq7RAMpZCG8VSYfNImVeSiZNMb-EY_1tqg_z5A3fqTLNoudkhL3MkcxSomPV8iDpzsvkttkLG2TrHZ11havzc-RbRr9K4jVqUBRONWk44S_MjqU3Wc3Vpc7TFIUnSPKcPySDx7HJlhvQhCIKq33hJudnR9v7gbWQuFV3nbJO9949qP16-uxDjTyL824hJhViv71vF9OsRZjVi3bsrwvzrERY1Ihl8OgJ4sFUpB2pUiR4UqAp2GcLpQo7B7LDAGKjtcqayGGJxtp49E3-kLGqbkuV2BzRhubajcs_twZ2kbKtY8TarUHBUY1kKA7TnOwbQExyAtvth0w96Hb8ftv3uoBt6cAQahM1W2O763U47bA3CTu_swav7aavZ73U9QJ7aq_Hz-nq4R-T8H-HTa5s)

In [ ]:
# Install Dependencies:
# !pip install -q google-adk google-genai kaggle_secrets

# Imports
import os
import json
from datetime import datetime, timedelta
from typing import List, Dict, Any

# API Key
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
GOOGLE_API_KEY = secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# ADK
from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.apps.app import App
from google.adk.tools.tool_context import ToolContext
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai import types
from google.genai.types import Content, Part

print("Imports Initialized.")

## Architecture

The project leverages the **Google AI Developer Kit (ADK)** to implement a multi-agent workflow:

### 1. **Task Intake Agent**
   - Parses free-text task descriptions from the user.
   - Outputs structured JSON containing task title, duration, optional preferred start time, and type.


### 2. **Schedule Generator Agent**
   - Reads tasks from session state.
   - Uses a custom **TimeConstraintTool** to enforce daily start/end windows.
   - Produces a readable schedule text.


### 3. **Sequential Agent**
   - Combines the two agents into a single workflow.
   - Ensures tasks flow from intake to scheduling in an organized manner.

In [ ]:
# Utility Function
def hhmm_to_minutes(hhmm_string: str):
    """
    Convert 'HH:MM' into minutes since midnight.
    Safe, simple, and used by the scheduling tool.
    """
    if not isinstance(hhmm_string, str):
        print("Error: preferred_start must be a string like '09:30'")
        return None

    try:
        # Try parsing HH:MM format
        t = datetime.strptime(hhmm_string, "%H:%M")
        return t.hour * 60 + t.minute

    except ValueError:
        print(f"Error: Invalid time format '{hhmm_string}', expected HH:MM.")
        return None


# Custom TimeConstraintTool
def time_constraint_tool(tasks: List[Dict[str, Any]],
                         start_window: str = "07:00",
                         end_window: str = "22:00",
                         tool_context: ToolContext = None) -> Dict[str, Any]:
    """
    Adjust tasks to fit inside a daily scheduling window.
    """

    # Convert window boundaries to time objects
    win_start = datetime.strptime(start_window, "%H:%M").time()
    win_end   = datetime.strptime(end_window, "%H:%M").time()

    today = datetime.today().date()
    cursor = datetime.combine(today, win_start)  # where scheduling begins

    adjusted = []

    for task in tasks:
        duration = int(task.get("duration_minutes", 30))

        pref = task.get("preferred_start")

        # If preferred_start exists, try to use it
        if pref:
            pref_minutes = hhmm_to_minutes(pref)
            if pref_minutes is not None:
                pref_time = (datetime.combine(today, datetime.min.time()) +
                             timedelta(minutes=pref_minutes)).time()

                # Only respect preferred time if it’s inside allowed window
                if win_start <= pref_time <= win_end:
                    cursor = datetime.combine(today, pref_time)

        # Calculate task end time
        end_dt = cursor + timedelta(minutes=duration)

        # If end time exceeds daily window, push task to next day at start_window
        if end_dt.time() > win_end:
            cursor = datetime.combine(cursor.date() + timedelta(days=1), win_start)
            end_dt = cursor + timedelta(minutes=duration)

        # Save task
        adjusted.append({
            "title": task.get("title"),
            "type": task.get("type", "task"),
            "start": cursor.strftime("%Y-%m-%d %H:%M"),
            "end":   end_dt.strftime("%Y-%m-%d %H:%M"),
            "duration_minutes": duration,
        })

        # Move cursor for next task
        cursor = end_dt

    # ADK Tool: save results into state if allowed
    if tool_context:
        tool_context.state["adjusted_tasks"] = adjusted

    return {"adjusted_tasks": adjusted}


print("Tools Initialized.")

In [ ]:
# Testing the tool
sample_tasks = [
    {"title": "Math Assignment", "duration_minutes": 90, "preferred_start": "09:00"},
    {"title": "Lab Meeting", "duration_minutes": 60, "preferred_start": "13:00"},
    {"title": "Gym", "duration_minutes": 45},
    {"title": "Evening Study", "duration_minutes": 120, "preferred_start": "18:00"},
]

result = time_constraint_tool(sample_tasks)

print("=== Adjusted Tasks ===")
for t in result["adjusted_tasks"]:
    print(f"{t['start']} -> {t['end']} | {t['title']}")

## Components

### 1. Custom Tool: TimeConstraintTool
- Enforces start and end windows for daily tasks.
- Adjusts task start/end times to fit within user-defined working hours.
- Updates session state with adjusted tasks for further processing.

### 2. Sessions & Memory
- Uses **InMemorySessionService** to store intermediate states.
- Tasks, adjusted schedules, and readable summaries are stored in session state.
- Supports reproducibility and allows for resuming previous sessions.

### 3. Agents
- **Task Intake Agent**: Converts free-text task input into structured JSON.
- **Schedule Generator Agent**: Produces a human-readable schedule using the adjusted tasks.
- **Sequential Agent**: Orchestrates the flow between agents.

In [ ]:
# Agent Instructions
task_intake_instructions = """
You are the Task Intake Agent. 
Input: A free-text list of tasks/events provided by the user.

Output Format:
Return ONLY a JSON array (a list).
Do NOT wrap it inside any object.
Each item must contain:
- "title": string
- "duration_minutes": integer
- "preferred_start": string in "HH:MM" format (only if mentioned)
- "type": either "task" or "event"

Correct Example Output:
[
  {"title": "Gym", "duration_minutes": 60, "type": "task"},
  {"title": "Dinner", "duration_minutes": 45, "preferred_start": "19:00", "type": "event"}
]
]
"""

schedule_generator_instructions = """
You are the Schedule Generator Agent. 
1. You will receive a list of 'tasks' in the state.
2. You MUST call the 'time_constraint_tool' passing these tasks to calculate specific start/end times.
3. Based on the output of the tool (adjusted_tasks), write a friendly text summary of the day.
"""

print("Instructions Initialized.")

In [ ]:
# Schema for Structured Output
from pydantic import BaseModel, Field
from typing import Optional, List

class Task(BaseModel):
    title: str = Field(..., description="Short task name")
    description: Optional[str] = Field(None, description="Extra task details")
    due_date: Optional[str] = Field(None, description="ISO date if applicable")
    duration_minutes: Optional[int] = Field(None, description="Estimated duration in minutes")
    priority: Optional[str] = Field(None, description="low / medium / high")

class TaskList(BaseModel):
    tasks: List[Task] = Field(default_factory=list)

print("Schema Initialized.")

In [ ]:
# LLM Agents
task_intake_agent = LlmAgent(
    name="TaskIntakeAgent",
    model="gemini-2.5-flash-lite",
    instruction=task_intake_instructions,
    description="Parses free-form user input into structured task objects.",
    output_schema=TaskList,            # Schema
    output_key="tasks"
)

schedule_generator_agent = LlmAgent(
    name="ScheduleGeneratorAgent",
    model="gemini-2.5-flash-lite",
    instruction=schedule_generator_instructions,
    description="Generates a human-readable daily schedule using tool-processed tasks.",
    output_key="readable_schedule",
    tools=[time_constraint_tool]
)

# Sequential Pipeline
pipeline = SequentialAgent(
    name="SmartDailySchedulerPipeline",
    sub_agents=[task_intake_agent, schedule_generator_agent],
    description="Main pipeline linking Task Intake → Tool → Schedule Generation."
)

# Create  App
pipeline_app = App(
    name="SmartDailySchedulerApp",
    root_agent=pipeline,
)

session_service = InMemorySessionService()

# Create runner with the App
pipeline_runner = Runner(
    app=pipeline_app,  # Pass the App instead of the agent
    session_service=session_service,
)

print("LLM Agents Initialized.")

## Demonstrated ADK Concepts

### 1. **Multi-Agent System**
   - Multiple agents (Task Intake & Schedule Generator) collaborate to achieve a complete workflow.

### 2. **Custom Tools**
   - TimeConstraintTool demonstrates how to extend agent capabilities beyond standard LLM reasoning.

### 3. **Sessions & Memory**
   - InMemorySessionService shows how agents can store and retrieve structured data during a session.
   - Enables stateful computation and reuse of previous outputs.

This project serves as a clear, small-scale example of building a stateful multi-agent AI workflow using Google ADK.

## How It Works: Step-by-Step

The Smart Daily Scheduler Agent workflow can be summarized in the following steps:

---

### 1. User Input
The user provides a free-text list of tasks, events, or deadlines with optional preferred start times and durations.  

**Example Input:**
* Finish Math assignment, 90 minutes, start around 09:00
* Lab meeting, 60 minutes, preferred at 13:00
* Gym, 45 minutes
* Study for SAT exam, 120 minutes, preferred 18:00
* Sleep, 480 minutes, preferred 23:00

---

### 2. Task Intake Agent
- Converts the free-text input into a structured JSON array.
- Each task has:
  - `title` (string)
  - `duration_minutes` (integer)
  - `preferred_start` (optional "HH:MM")
  - `type` ("task" or "event")  

**Output Example (JSON):**
```json
[
  {"title": "Finish Math assignment", "duration_minutes": 90, "preferred_start": "09:00", "type": "task"},
  {"title": "Lab meeting", "duration_minutes": 60, "preferred_start": "13:00", "type": "event"},
  {"title": "Gym", "duration_minutes": 45, "type": "task"}
]

---
## Verbose / Debug Cell

**Purpose:**  
This cell is used to debug the Smart Daily Scheduler pipeline by streaming and inspecting all intermediate events from the agents and tools. It allows developers to see exactly what the agents receive, how the tools process tasks, and how the session state evolves.

**Core Concepts Implemented:**

1. **Verbose Event Streaming:**  
   - Uses `run_async()` from ADK agents to stream events step-by-step.
   - Each event is inspected for:
     - `parts` text content
     - `candidates` returned by the model
     - `function_call` invocations
   - Provides detailed information on the LLM output and tool interactions.

2. **Session Management Insight:**  
   - Prints session creation/resumption status.
   - Shows session state keys at each stage.
   - Helps identify why certain outputs may be missing on first run.

3. **Unit Testing of Tools:**  
   - Directly calls `time_constraint_tool` on tasks stored in session state.
   - Verifies that scheduling logic works independently of the pipeline.

**Benefits:**  
- Pinpoints why the first run may have empty outputs.
- Identifies issues in agent instructions, session state handling, or tool invocation.
- Essential for debugging multi-agent pipelines with complex state transitions.


In [ ]:
# DEBUG / VERBOSE TEST CELL 
import asyncio, nest_asyncio, json, traceback
nest_asyncio.apply()

# Global User/Session IDs
USER_ID = "default_user"
SESSION = "default_session"

# Helper: pretty-print an ADK event safely
def inspect_event(i, event):
    print(f"\n--- EVENT #{i} --- type={type(event)}")
    try:
        if hasattr(event, "text") and event.text:
            print("event.text:", event.text)
        if hasattr(event, "parts") and event.parts:
            for idx, p in enumerate(event.parts):
                print(f"part[{idx}].text:", getattr(p, "text", None))
        if hasattr(event, "content") and getattr(event, "content", None):
            try:
                parts = getattr(event.content, "parts", None)
                if parts:
                    for idx, p in enumerate(parts):
                        print(f"content.parts[{idx}].text:", getattr(p,"text",None))
            except Exception as e:
                print("  (reading content.parts failed)", e)
        if hasattr(event, "function_call") and getattr(event, "function_call", None):
            print("function_call:", event.function_call)
    except Exception as e:
        print("inspect_event error:", e)
        traceback.print_exc()

# Main debug run
async def debug_run_verbose():
    try:
        # 1) Create or get session
        try:
            session = await session_service.create_session(
                app_name=pipeline_app.name,
                user_id=USER_ID,
                session_id=SESSION
            )
            created = True
        except Exception:
            session = await session_service.get_session(
                app_name=pipeline_app.name,
                user_id=USER_ID,
                session_id=SESSION
            )
            created = False
        print(f"Session ready. id={session.id} (created={created})")
        print("Initial session.state keys:", list(session.state.keys()))

        # 2) Send user input (Task Intake)
        user_text = """
Please parse these tasks:
- Finish Math assignment, 90 minutes, preferred start 09:00
- Lab meeting, 60 minutes, preferred start 13:00
- Gym, 45 minutes
- Study SAT Exam, 120 minutes, preferred start 18:00
"""
        class UMsg:
            def __init__(self, text):
                self.role = "user"
                self.parts = [type("Part", (), {"text": text})()]

        umsg = UMsg(user_text)

        # 3) Run the full pipeline with Runner
        print("\n=== Streaming events from pipeline_runner.run_async ===")
        i = 0
        async for event in pipeline_runner.run_async(
            user_id=USER_ID,
            session_id=session.id,
            new_message=umsg
        ):
            inspect_event(i, event)
            i += 1

        # 4) Test time_constraint_tool directly if tasks exist
            tasks_dict = session.state.get("tasks", {})
            tasks_list = tasks_dict.get("tasks", [])  # Extract the actual list
            
            if tasks_list:
                print("\n--- Running time_constraint_tool directly as a unit test ---")
            try:
                tool_result = time_constraint_tool(tasks_list, start_window="07:00", end_window="23:00", tool_context=None)
                print("Tool returned:", json.dumps(tool_result, indent=2))
            except Exception as e:
                print("time_constraint_tool raised:", e)
                traceback.print_exc()
        else:
            print("\nNo tasks found in state after Task Intake. Skipping tool unit test.")

        # 5) Final session state inspection
        print("\n--- FINAL SESSION STATE ---")
        print("session.state keys:", list(session.state.keys()))
        print("tasks:", json.dumps(session.state.get("tasks", None), indent=2))
        print("adjusted_tasks:", json.dumps(session.state.get("adjusted_tasks", None), indent=2))
        print("readable_schedule:", session.state.get("readable_schedule", None))

    except Exception as e:
        print("DEBUG RUN EXCEPTION:", e)
        traceback.print_exc()

# Run the debug
print("Verbose Initialized.")
# await debug_run_verbose() # commented to preserve tokens

## Run Helper Function

**Purpose:**  
Simplifies running the Smart Daily Scheduler pipeline. Accepts a user input text, triggers the Task Intake Agent and Schedule Generator Agent, and returns structured outputs including tasks, adjusted tasks, and a human-readable schedule.

### **Core Concepts Implemented:**

1. **Session Management:**  
   - Checks whether the session already exists.
   - Creates a new session if it doesn't exist.
   - Ensures that outputs are populated even on the **first run**.

2. **Pipeline Execution:**  
   - Runs the full pipeline using `pipeline_runner.run_async()`.
   - Does not manually call tools; relies on the pipeline to invoke the `time_constraint_tool`.

3. **Output Retrieval:**  
   - Returns:
     - Raw tasks parsed by Task Intake Agent
     - Adjusted tasks processed by the scheduling tool
     - Readable, human-friendly daily schedule

### **Benefits:**  
- Eliminates manual pipeline management for the user.
- Guarantees that scheduling works correctly on the first run.
- Provides a clean, formatted result for downstream use or display.

In [ ]:
# === RUN HELPER FOR PIPELINE ===
import asyncio, nest_asyncio
nest_asyncio.apply()

# Global session/user constants
USER_ID = "default"
SESSION = "default"

# Helper to wrap user messages
class UserMessage:
    def __init__(self, text: str):
        self.role = "user"
        self.parts = [type("Part", (), {"text": text})()]

async def run_helper(user_text: str, session_id=SESSION, user_id=USER_ID):
    """
    Run the Smart Daily Scheduler pipeline for a given user input.
    Keeps session persistent.
    """
    # 1) Create or get session
    try:
        session = await session_service.create_session(
            app_name=pipeline_app.name,
            user_id=user_id,
            session_id=session_id
        )
        created = True
    except Exception:
        session = await session_service.get_session(
            app_name=pipeline_app.name,
            user_id=user_id,
            session_id=session_id
        )
        created = False

    # 2) Wrap user input
    msg = UserMessage(user_text)

    # 3) Run pipeline (Task Intake + Schedule Generator)
    async for _ in pipeline_runner.run_async(user_id=user_id, session_id=session.id, new_message=msg):
        pass  # consume generator

    # 4) Retrieve outputs
    tasks_dict = session.state.get("tasks", {})
    tasks_list = tasks_dict.get("tasks", [])

    if tasks_list:
        tool_result = time_constraint_tool(tasks_list, start_window="07:00", end_window="23:00", tool_context=None)
    else:
        tool_result = {"adjusted_tasks": []}

    adjusted_tasks = tool_result.get("adjusted_tasks", [])
    readable_schedule = session.state.get("readable_schedule", "No schedule generated.")

    return {
        "session": session,
        "adjusted_tasks": adjusted_tasks,
        "readable_schedule": readable_schedule
    }


print("Run Helper Initialized.")

## Interactive User Input Cell

**Purpose:**  
Allows the user to interact with the Smart Daily Scheduler in the same session. New task lists or schedule requests can be sent without creating a new session, maintaining context across multiple interactions.

### **Core Concepts Implemented:**

1. **Persistent Session State:**  
   - Maintains all previous tasks, adjusted schedules, and conversation history in the same session.
   - Supports incremental updates and repeated scheduling without data loss.

2. **Simplified User Input:**  
   - Accepts free-form text from the user.
   - Automatically triggers Task Intake and Schedule Generator agents in the pipeline.

3. **Formatted Outputs:**  
   - Returns updated raw tasks, adjusted tasks, and a human-readable schedule.
   - Ensures clarity and usability for end users.

### **Benefits:**  
- Provides a true “interactive assistant” experience.
- Avoids the first-run issue by always relying on a managed session.
- Keeps user workflow consistent, even across multiple inputs.


In [ ]:
# === INTERACTIVE CONVERSATION WITH SAME SESSION ===
import json

# Example usage:
user_input = """
Tasks for Thursday:
- Finish Math assignment, 90 minutes, preferred start 09:00
- Lab meeting, 60 minutes, preferred start 13:00
- Gym, 45 minutes
- Study for SAT Exam, 120 minutes, preferred start 18:00
- Sleep by 23:00
"""

# Run the helper and get output
out = await run_helper(user_input)

# Print outputs nicely
print("\n=== Adjusted Tasks ===")
for t in out['adjusted_tasks']:
    print(f"* {t['start']} → {t['end']} | [{t['type']}] {t['title']} ({t['duration_minutes']} mins)")

print("\n=== Readable Schedule ===")
print(out['readable_schedule'])

# Store the session object to keep interacting in the same session
current_session = out['session']

---